## व्यंजन वर्गीकरणकर्ता 2

इस दूसरे वर्गीकरण पाठ में, हम श्रेणीबद्ध डेटा को वर्गीकृत करने के `अधिक तरीकों` का पता लगाएंगे। हम यह भी जानेंगे कि एक वर्गीकरणकर्ता को दूसरे के ऊपर चुनने के क्या परिणाम हो सकते हैं।

### [**पाठ से पहले का क्विज़**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **पूर्वापेक्षा**

हम मानते हैं कि आपने पिछले पाठ पूरे कर लिए हैं क्योंकि हम पहले सीखे गए कुछ अवधारणाओं को आगे बढ़ाएंगे।

इस पाठ के लिए, हमें निम्नलिखित पैकेजों की आवश्यकता होगी:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) एक [R पैकेजों का संग्रह](https://www.tidyverse.org/packages) है, जिसे डेटा विज्ञान को तेज़, आसान और अधिक मज़ेदार बनाने के लिए डिज़ाइन किया गया है!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) फ्रेमवर्क मॉडलिंग और मशीन लर्निंग के लिए [पैकेजों का संग्रह](https://www.tidymodels.org/packages/) है।

-   `themis`: [themis पैकेज](https://themis.tidymodels.org/) असंतुलित डेटा से निपटने के लिए अतिरिक्त रेसिपी स्टेप्स प्रदान करता है।

आप इन्हें इस प्रकार इंस्टॉल कर सकते हैं:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

वैकल्पिक रूप से, नीचे दिया गया स्क्रिप्ट यह जांचता है कि क्या आपके पास इस मॉड्यूल को पूरा करने के लिए आवश्यक पैकेज हैं और यदि वे गायब हैं तो उन्हें आपके लिए इंस्टॉल करता है।


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. एक वर्गीकरण मानचित्र**

हमारे [पिछले पाठ](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) में, हमने इस सवाल का जवाब देने की कोशिश की: हम कई मॉडलों में से कैसे चुनें? काफी हद तक, यह डेटा की विशेषताओं और उस समस्या के प्रकार पर निर्भर करता है जिसे हम हल करना चाहते हैं (जैसे कि वर्गीकरण या प्रतिगमन?)

पहले, हमने माइक्रोसॉफ्ट के चीट शीट का उपयोग करके डेटा को वर्गीकृत करने के विभिन्न विकल्पों के बारे में सीखा। पायथन के मशीन लर्निंग फ्रेमवर्क, Scikit-learn, एक समान लेकिन अधिक विस्तृत चीट शीट प्रदान करता है, जो आपके अनुमानों (classifiers का एक और नाम) को और अधिक सीमित करने में मदद कर सकता है:

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> टिप: [इस मानचित्र को ऑनलाइन देखें](https://scikit-learn.org/stable/tutorial/machine_learning_map/) और रास्ते पर क्लिक करके प्रलेखन पढ़ें।  
>  
> [Tidymodels संदर्भ साइट](https://www.tidymodels.org/find/parsnip/#models) विभिन्न प्रकार के मॉडलों के बारे में उत्कृष्ट प्रलेखन भी प्रदान करती है।  

### **योजना** 🗺️

यह मानचित्र तब बहुत सहायक होता है जब आपको अपने डेटा की स्पष्ट समझ हो, क्योंकि आप इसके रास्तों पर चलते हुए निर्णय तक पहुँच सकते हैं:

-   हमारे पास \>50 नमूने हैं  

-   हम एक श्रेणी की भविष्यवाणी करना चाहते हैं  

-   हमारे पास लेबल किया हुआ डेटा है  

-   हमारे पास 100K से कम नमूने हैं  

-   ✨ हम एक Linear SVC चुन सकते हैं  

-   यदि यह काम नहीं करता है, क्योंकि हमारे पास संख्यात्मक डेटा है  

    -   हम ✨ KNeighbors Classifier आज़मा सकते हैं  

        -   यदि यह भी काम नहीं करता है, तो ✨ SVC और ✨ Ensemble Classifiers आज़माएँ  

यह एक बहुत ही सहायक मार्गदर्शिका है। अब, चलिए इसे [tidymodels](https://www.tidymodels.org/) मॉडलिंग फ्रेमवर्क का उपयोग करके शुरू करते हैं: R पैकेजों का एक सुसंगत और लचीला संग्रह, जिसे अच्छे सांख्यिकीय अभ्यास को प्रोत्साहित करने के लिए विकसित किया गया है 😊।  

## 2. डेटा को विभाजित करें और असंतुलित डेटा सेट को संभालें।  

हमने अपने पिछले पाठों में सीखा कि हमारी व्यंजनों में कुछ सामान्य सामग्री थीं। साथ ही, व्यंजनों की संख्या में काफी असमान वितरण था।  

हम इनसे निपटेंगे:  

-   उन सबसे सामान्य सामग्रियों को हटाकर जो अलग-अलग व्यंजनों के बीच भ्रम पैदा करती हैं, `dplyr::select()` का उपयोग करके।  

-   एक `recipe` का उपयोग करके जो डेटा को पूर्व-प्रसंस्करण करता है और इसे मॉडलिंग के लिए तैयार करता है, एक `over-sampling` एल्गोरिदम लागू करके।  

हमने उपरोक्त को पिछले पाठ में पहले ही देखा है, इसलिए यह आसान होना चाहिए 🥳!  


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### असंतुलित डेटा से निपटना

असंतुलित डेटा अक्सर मॉडल के प्रदर्शन पर नकारात्मक प्रभाव डालता है। कई मॉडल तब सबसे अच्छा प्रदर्शन करते हैं जब अवलोकनों की संख्या समान होती है, और इसलिए असंतुलित डेटा के साथ संघर्ष करते हैं।

असंतुलित डेटा सेट से निपटने के दो मुख्य तरीके हैं:

-   अल्पसंख्यक वर्ग में अवलोकन जोड़ना: `ओवर-सैंपलिंग`, जैसे SMOTE एल्गोरिदम का उपयोग करना, जो इन मामलों के निकटतम पड़ोसियों का उपयोग करके अल्पसंख्यक वर्ग के नए उदाहरण कृत्रिम रूप से उत्पन्न करता है।

-   बहुसंख्यक वर्ग से अवलोकन हटाना: `अंडर-सैंपलिंग`

हमने अपने पिछले पाठ में दिखाया था कि कैसे एक `recipe` का उपयोग करके असंतुलित डेटा सेट से निपटा जा सकता है। एक recipe को एक खाका के रूप में सोचा जा सकता है जो यह वर्णन करता है कि डेटा सेट पर कौन-कौन से कदम लागू किए जाने चाहिए ताकि इसे डेटा विश्लेषण के लिए तैयार किया जा सके। हमारे मामले में, हम चाहते हैं कि हमारे `training set` में हमारे व्यंजनों की संख्या का समान वितरण हो। चलिए इसे शुरू करते हैं।


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

अब हम मॉडल्स को प्रशिक्षित करने के लिए तैयार हैं 👩‍💻👨‍💻!

## 3. बहु-नामांकित प्रतिगमन मॉडल्स से आगे

हमने अपने पिछले पाठ में बहु-नामांकित प्रतिगमन मॉडल्स पर चर्चा की थी। आइए अब वर्गीकरण के लिए कुछ और लचीले मॉडल्स का अन्वेषण करें।

### सपोर्ट वेक्टर मशीन्स

वर्गीकरण के संदर्भ में, `सपोर्ट वेक्टर मशीन्स` एक मशीन लर्निंग तकनीक है जो *हाइपरप्लेन* खोजने की कोशिश करती है, जो कक्षाओं को "सबसे अच्छे" तरीके से अलग करता है। आइए एक सरल उदाहरण देखें:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ कक्षाओं को अलग नहीं करता है। H2~ अलग करता है, लेकिन केवल एक छोटे अंतर के साथ। H3~ उन्हें अधिकतम अंतर के साथ अलग करता है।

#### रेखीय सपोर्ट वेक्टर क्लासिफायर

सपोर्ट-वेक्टर क्लस्टरिंग (SVC) मशीन लर्निंग तकनीकों के सपोर्ट-वेक्टर मशीन परिवार का एक हिस्सा है। SVC में, हाइपरप्लेन को इस तरह चुना जाता है कि वह `अधिकांश` प्रशिक्षण अवलोकनों को सही ढंग से अलग कर सके, लेकिन `कुछ अवलोकनों को गलत वर्गीकृत` कर सकता है। कुछ बिंदुओं को गलत पक्ष पर रहने की अनुमति देकर, SVM बाहरी मूल्यों (outliers) के प्रति अधिक मजबूत बन जाता है और इस प्रकार नए डेटा पर बेहतर सामान्यीकरण करता है। इस उल्लंघन को नियंत्रित करने वाले पैरामीटर को `cost` कहा जाता है, जिसकी डिफ़ॉल्ट मान 1 होती है (देखें `help("svm_poly")`)।

आइए एक रेखीय SVC बनाएं, इसके लिए बहुपद SVM मॉडल में `degree = 1` सेट करें।


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

अब जब हमने प्रीप्रोसेसिंग चरणों और मॉडल विनिर्देश को *workflow* में कैद कर लिया है, तो हम आगे बढ़ सकते हैं और linear SVC को प्रशिक्षित कर सकते हैं और साथ ही परिणामों का मूल्यांकन कर सकते हैं। प्रदर्शन मेट्रिक्स के लिए, आइए एक मेट्रिक सेट बनाएं जो इनका मूल्यांकन करेगा: `accuracy`, `sensitivity`, `Positive Predicted Value` और `F Measure`।

> `augment()` दिए गए डेटा में भविष्यवाणियों के लिए कॉलम जोड़ देगा।


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### सपोर्ट वेक्टर मशीन

सपोर्ट वेक्टर मशीन (SVM) सपोर्ट वेक्टर क्लासिफायर का विस्तार है, जो वर्गों के बीच एक गैर-रेखीय सीमा को समायोजित करने के लिए बनाया गया है। मूल रूप से, SVMs *कर्नल ट्रिक* का उपयोग करते हैं ताकि फीचर स्पेस को बड़ा किया जा सके और वर्गों के बीच गैर-रेखीय संबंधों के अनुकूल बनाया जा सके। SVMs द्वारा उपयोग किया जाने वाला एक लोकप्रिय और अत्यधिक लचीला कर्नल फ़ंक्शन *रेडियल बेसिस फ़ंक्शन* है। आइए देखें कि यह हमारे डेटा पर कैसा प्रदर्शन करता है।


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

बहुत बेहतर 🤩!

> ✅ कृपया देखें:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> अधिक पढ़ने के लिए।

### निकटतम पड़ोसी वर्गीकरणकर्ता

*K*-nearest neighbor (KNN) एक एल्गोरिथ्म है जिसमें प्रत्येक अवलोकन को अन्य अवलोकनों के साथ उसकी *समानता* के आधार पर भविष्यवाणी की जाती है।

आइए इसे हमारे डेटा पर लागू करें।


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

ऐसा लगता है कि यह मॉडल उतना अच्छा प्रदर्शन नहीं कर रहा है। संभवतः मॉडल के तर्कों को बदलने से (देखें `help("nearest_neighbor")`) मॉडल का प्रदर्शन बेहतर हो सकता है। इसे आज़माना सुनिश्चित करें।

> ✅ कृपया देखें:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> अधिक जानने के लिए *K*-Nearest Neighbors classifiers के बारे में।

### Ensemble classifiers

Ensemble एल्गोरिदम कई बेस अनुमानों को मिलाकर एक बेहतर मॉडल बनाने का काम करते हैं, जो निम्नलिखित तरीकों से किया जा सकता है:

`bagging`: बेस मॉडल के संग्रह पर *औसत निकालने का कार्य* लागू करना

`boosting`: ऐसे मॉडल की श्रृंखला बनाना जो एक-दूसरे पर आधारित होते हैं ताकि भविष्यवाणी प्रदर्शन को बेहतर बनाया जा सके।

चलो शुरुआत करते हैं एक Random Forest मॉडल को आज़माने से, जो निर्णय वृक्षों का एक बड़ा संग्रह बनाता है और फिर एक औसत निकालने का कार्य लागू करता है ताकि एक बेहतर समग्र मॉडल तैयार हो सके।


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

अच्छा काम 👏!

आइए अब Boosted Tree मॉडल के साथ भी प्रयोग करें।

Boosted Tree एक ensemble विधि को परिभाषित करता है, जो एक श्रृंखला में क्रमिक निर्णय पेड़ों का निर्माण करता है, जहां प्रत्येक पेड़ पिछले पेड़ों के परिणामों पर निर्भर करता है ताकि त्रुटि को धीरे-धीरे कम किया जा सके। यह गलत वर्गीकृत वस्तुओं के वज़न पर ध्यान केंद्रित करता है और अगले classifier के लिए फिट को समायोजित करता है ताकि सुधार किया जा सके।

इस मॉडल को फिट करने के अलग-अलग तरीके हैं (देखें `help("boost_tree")`)। इस उदाहरण में, हम Boosted trees को `xgboost` इंजन के माध्यम से फिट करेंगे।


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ कृपया देखें:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - AdaBoost मॉडल की खोज करता है, जो xgboost का एक अच्छा विकल्प है।
>
> अधिक जानने के लिए Ensemble classifiers के बारे में।

## 4. अतिरिक्त - कई मॉडलों की तुलना करना

हमने इस लैब में काफी सारे मॉडल फिट किए हैं 🙌। अलग-अलग प्रीप्रोसेसर और/या मॉडल स्पेसिफिकेशन के सेट से कई वर्कफ़्लो बनाना और फिर प्रदर्शन मेट्रिक्स को एक-एक करके गणना करना थकाऊ या कठिन हो सकता है।

आइए देखें कि क्या हम इसे एक फ़ंक्शन बनाकर हल कर सकते हैं, जो ट्रेनिंग सेट पर वर्कफ़्लो की एक सूची फिट करता है और फिर टेस्ट सेट के आधार पर प्रदर्शन मेट्रिक्स लौटाता है। हम [purrr](https://purrr.tidyverse.org/) पैकेज से `map()` और `map_dfr()` का उपयोग करेंगे ताकि सूची के प्रत्येक तत्व पर फ़ंक्शन लागू कर सकें।

> [`map()`](https://purrr.tidyverse.org/reference/map.html) फ़ंक्शन आपको कई for loops को कोड से बदलने की अनुमति देता है, जो अधिक संक्षिप्त और पढ़ने में आसान होता है। [`map()`](https://purrr.tidyverse.org/reference/map.html) फ़ंक्शन के बारे में सीखने के लिए सबसे अच्छा स्थान R for data science की [iteration chapter](http://r4ds.had.co.nz/iteration.html) है।


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) पैकेज उपयोगकर्ताओं को बड़ी संख्या में मॉडल बनाने और आसानी से फिट करने की अनुमति देता है, लेकिन यह मुख्य रूप से `cross-validation` जैसी पुनःनमूना तकनीकों के साथ काम करने के लिए डिज़ाइन किया गया है, जिसे हम अभी कवर करना बाकी है।

## **🚀चुनौती**

इन तकनीकों में से प्रत्येक में कई पैरामीटर होते हैं जिन्हें आप समायोजित कर सकते हैं, जैसे कि SVMs में `cost`, KNN में `neighbors`, और रैंडम फॉरेस्ट में `mtry` (रैंडमली चुने गए प्रेडिक्टर्स)।

प्रत्येक के डिफ़ॉल्ट पैरामीटर पर शोध करें और सोचें कि इन पैरामीटर को समायोजित करने से मॉडल की गुणवत्ता पर क्या प्रभाव पड़ेगा।

किसी विशेष मॉडल और उसके पैरामीटर के बारे में अधिक जानने के लिए, उपयोग करें: `help("model")` जैसे `help("rand_forest")`

> व्यवहार में, हम आमतौर पर *सर्वश्रेष्ठ मान* का *अनुमान* लगाने के लिए कई मॉडल को एक `सिम्युलेटेड डेटा सेट` पर प्रशिक्षित करते हैं और मापते हैं कि ये सभी मॉडल कितनी अच्छी तरह प्रदर्शन करते हैं। इस प्रक्रिया को **ट्यूनिंग** कहा जाता है।

### [**पोस्ट-लेक्चर क्विज़**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **समीक्षा और स्व-अध्ययन**

इन पाठों में बहुत सारे तकनीकी शब्द हैं, इसलिए [इस सूची](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) को देखने के लिए कुछ समय निकालें, जिसमें उपयोगी शब्दावली दी गई है!

#### धन्यवाद:

[`एलिसन हॉर्स्ट`](https://twitter.com/allison_horst/) को, जिन्होंने अद्भुत चित्रण बनाए हैं जो R को अधिक स्वागतपूर्ण और आकर्षक बनाते हैं। उनके और चित्रण [गैलरी](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM) में देखें।

[Cassie Breviu](https://www.twitter.com/cassieview) और [Jen Looper](https://www.twitter.com/jenlooper) को, जिन्होंने इस मॉड्यूल का मूल Python संस्करण बनाया ♥️

खुशहाल सीखने की शुभकामनाएं,

[Eric](https://twitter.com/ericntay), गोल्ड माइक्रोसॉफ्ट लर्न स्टूडेंट एंबेसडर।

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>चित्रण @allison_horst द्वारा</figcaption>



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
